<a href="https://colab.research.google.com/github/custom-hyper/DeFi_Lama_API/blob/master/DeFi_Lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_#Install Dependencies

In [1]:
 from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install requests
!pip install json


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


#Extract All Protocols 

Please use  df['slug'] as primary key if loaded into SQL light

In [4]:
#@title Historical TVL { display-mode: "code" }
import requests
import json
from google.colab import data_table
import pandas as pd
import logging
import pandas as pd
import logging
import time
from tqdm.notebook import tqdm
from datetime import datetime


#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
table = json.loads(output)
df = pd.json_normalize(table)
slug = df['slug'][:3]

path = "https://api.llama.fi/protocol/"
frames = []


try:
  for i in tqdm(slug):
    response = requests.get(path + i)
    output = response.text
    DeFi = json.loads(output)

    full_table = pd.json_normalize(DeFi['tvl']) 
    full_table['slug'] = i
    

    #results = [full_table]
    frames.append(full_table)

    #Slow down requests
    
    time.sleep(60/50)
    #Status report
    
    
except ConnectionError:
    time.sleep(60)


single = pd.concat(frames) #Careful, this line must be outside of the for loop
#single['date_Standard']  = datetime.fromtimestamp(time).strftime('%Y-%m-%d')
single = pd.DataFrame(single, index= ['slug'])
unique = single['slug'].unique()


for i in unique:
 
   single['totalLiquidity_percent_1d_USD'] = single['slug'].loc[i].pct_change(periods = 1)
   #single['totalLiquidity_percent_7d_USD'] = single['totalLiquidityUSD'].pct_change(periods = 7 )


path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output/"
single.to_csv( path + 'DeFi_Lama_tvl.csv')
  


  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: ignored

In [12]:
single.head()

,date,totalLiquidityUSD,slug
0,1581206400,1.163733e+06,curve
1,1581292800,1.372110e+06,curve
2,1581379200,9.607534e+05,curve
3,1581465600,1.006408e+06,curve
4,1581552000,1.258776e+06,curve


In [13]:
single.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1614 entries, 0 to 350
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               1614 non-null   int64  
 1   totalLiquidityUSD  1614 non-null   float64
 2   slug               1614 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 115.0+ KB


In [6]:
#@title Current Data { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table

import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt



#General Information table
response = requests.get("https://api.llama.fi/protocols")
output = response.text
DeFi = json.loads(output)
full_talbe = pd.json_normalize(DeFi)
full_talbe['id'] = full_talbe['symbol']  + '.' + full_talbe['name']
full_talbe['id'].replace(' ','')
full_talbe['Notes'] = ''

#UNIX timestamp: created
full_talbe['UNIX'] = int( time.time())

#CoinGecko webpage

 
path = "/content/drive/MyDrive/workspace/pipelines/datasets/defi_lama/output"
full_talbe.to_csv(path+'/Defi_llama_current.csv')
full_talbe.info()



,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,Notes,UNIX
0,CRV.Curve,Curve,0xD533a949740bb3306d119CC777fa900bA034cd52,CRV,https://curve.fi,Curve is a decentralized exchange liquidity po...,Multi-Chain,https://icons.llama.fi/curve.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1649304959
1,LDO.Lido,Lido,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,https://lido.fi/,"Liquidity for staked assets. Daily rewards, no...",Multi-Chain,https://icons.llama.fi/lido.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1649304959
2,ANC.Anchor,Anchor,-,ANC,https://anchorprotocol.com/,Money market on Terra,Multi-Chain,https://icons.llama.fi/anchor.jpg,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1649304959
3,MKR.MakerDAO,MakerDAO,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,MKR,https://makerdao.com/,"Builders of Dai, a digital currency that can b...",Ethereum,https://icons.llama.fi/makerdao.jpg,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1649304959
4,AAVE.AAVE,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,https://aave.com\r\n,Aave is an Open Source and Non-Custodial proto...,Multi-Chain,https://icons.llama.fi/aave.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1649304959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,AADA.Aada,Aada,cardano:asset1khk46tdfsknze9k84ae0ee0k2x8mcwhz...,AADA,https://aada.finance,This is Aada Finance - Decentralized Lending a...,Cardano,https://icons.llama.fi/aada.png,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1649304959
1455,PES.Pegasusdollar,Pegasusdollar,cronos:0x8efbaa6080412d7832025b03b9239d0be1e2aa3b,PES,https://pegasusdollar.finance/,The first algorithmic token pegged to METF run...,Cronos,https://icons.llama.fi/pegasusdollar.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1649304959
1456,PEAK.Peak Finance,Peak Finance,metis:0x1F5550A0F5F659E07506088A7919A88DfF37218f,PEAK,https://peakfinance.io,Peak Finance is a protocol designed to maintai...,Metis,https://icons.llama.fi/peak-finance.png,0,None,...,NaN,NaN,5.333388e+06,NaN,NaN,NaN,NaN,NaN,,1649304959
1457,REVU.Revuto,Revuto,cardano:asset14cvkz63kgtstwplh0n6hx52ezk0egg4f...,REVU,https://revuto.com,"Block, Snooze, or Approve any subscription pay...",Cardano,https://icons.llama.fi/revuto.png,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1649304959


In [7]:
full_talbe.describe()

,tvl,change_1h,change_1d,change_7d,staking,fdv,mcap,chainTvls.Optimism,chainTvls.Avalanche,chainTvls.Harmony,...,chainTvls.Arbitrum-offers,chainTvls.zkSync,chainTvls.zkSync-offers,chainTvls.Metis-pool2,chainTvls.Harmony-Treasury,chainTvls.Metis-Treasury,chainTvls.Velas-staking,chainTvls.Celo-Treasury,chainTvls.Ergo-staking,UNIX
count,1.459000e+03,1275.000000,1249.000000,1214.000000,6.610000e+02,6.230000e+02,1.025000e+03,3.100000e+01,1.900000e+02,4.900000e+01,...,1.000000,1.0,1.000000,3.000000e+00,1.000000,1.000000,1.000000,1.0,1.000000e+00,1.459000e+03
mean,1.801699e+08,-0.212787,-2.471886,44.735978,3.605685e+07,1.212360e+12,1.059201e+08,1.527615e+07,5.907681e+07,1.612587e+07,...,0.999871,0.0,314672.090376,1.799166e+06,0.000727,2773.580567,626899.365487,0.0,9.374468e+06,1.649305e+09
std,1.213508e+09,4.947115,25.180041,1536.870552,3.333987e+08,2.830783e+13,6.739114e+08,3.466313e+07,2.675220e+08,4.966404e+07,...,NaN,NaN,NaN,3.060764e+06,NaN,NaN,NaN,NaN,NaN,0.000000e+00
min,0.000000e+00,-100.000000,-100.000000,-100.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.999871,0.0,314672.090376,1.686550e+04,0.000727,2773.580567,626899.365487,0.0,9.374468e+06,1.649305e+09
25%,2.527929e+04,-0.168952,-5.730202,-11.233476,3.064422e+03,2.446794e+06,0.000000e+00,1.216569e+05,6.486440e+02,3.242260e+04,...,0.999871,0.0,314672.090376,3.205452e+04,0.000727,2773.580567,626899.365487,0.0,9.374468e+06,1.649305e+09
50%,1.274947e+06,0.000000,-3.510666,-5.473896,2.469583e+05,2.627958e+07,7.794810e+05,1.527655e+06,2.047760e+05,2.296716e+05,...,0.999871,0.0,314672.090376,4.724354e+04,0.000727,2773.580567,626899.365487,0.0,9.374468e+06,1.649305e+09
75%,1.875438e+07,0.079704,-0.741804,-0.248308,2.736620e+06,1.581642e+08,1.527780e+07,1.087561e+07,3.250080e+06,4.928952e+06,...,0.999871,0.0,314672.090376,2.690316e+06,0.000727,2773.580567,626899.365487,0.0,9.374468e+06,1.649305e+09
max,2.043576e+10,65.258317,499.711317,53214.081372,7.924902e+09,7.054630e+14,1.192485e+10,1.827352e+08,2.783485e+09,2.673399e+08,...,0.999871,0.0,314672.090376,5.333388e+06,0.000727,2773.580567,626899.365487,0.0,9.374468e+06,1.649305e+09


In [8]:
#@title Dataset Stats { display-mode: "code" }
full_talbe.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 297 columns):
 #    Column                            Dtype  
---   ------                            -----  
 0    id                                object 
 1    name                              object 
 2    address                           object 
 3    symbol                            object 
 4    url                               object 
 5    description                       object 
 6    chain                             object 
 7    logo                              object 
 8    audits                            object 
 9    audit_note                        object 
 10   gecko_id                          object 
 11   cmcId                             object 
 12   category                          object 
 13   chains                            object 
 14   module                            object 
 15   twitter                           object 
 16   audit_links           

In [11]:
  #@title Total Combined TVL { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table

import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

path = "https://api.llama.fi/charts"
#slug = 'aave' 
#General Information table
response = requests.get(path)
output = response.text
DeFi = json.loads(output)
totalLiq_talbe = pd.json_normalize(DeFi)
totalLiq_talbe



,date,totalLiquidityUSD
0,1541203200,3.468404e+04
1,1541289600,4.129299e+04
2,1541376000,4.155031e+04
3,1541462400,4.323356e+04
4,1541548800,1.538626e+05
...,...,...
1247,1648944000,2.319758e+11
1248,1649030400,2.332434e+11
1249,1649116800,2.337018e+11
1250,1649203200,2.285448e+11


# Feature Engineering